#### memory
> - 메모리 여러 개 -> 각자 저장 방식이 다르며, 각자만의 장단점이 있다.
> - 챗복에 메모리를 추가하지 않으면 챗복은 아무것도 기억할 수 없다. -> 이전 질문에 이어지는 질문을 기억하지 못 한다.
> - 메모리가 있어야 어떤 사람과 얘기하고 있다는 느낌을 들게 한다.

#### Conversation Buffer 메모리
> - 이전 대화 내용 전체를 저장하는 메모리
> - 대화가 길어질 수록 메모리도 계속 커져서 비효율적이다. / 이해하기 가장 쉬운 메모리
> - 메모리 종류와 무관하게 API는 다 똑같다.

In [3]:
# Conversation Buffer 메모리

from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True)

memory.save_context({"input": "Hi"}, {"output": "hello"})

memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi'), AIMessage(content='hello')]}

In [4]:
memory.save_context({"input": "Hi"}, {"output": "hello"})

memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi'),
  AIMessage(content='hello'),
  HumanMessage(content='Hi'),
  AIMessage(content='hello')]}

#### Conversation Buffer Window 메모리
> - 대화의 특정 부분, 가장 최근 부분만 저장하는 메모리 + 메모리 크기를 지정할 수 있다.
> - 챗봇이 전체 대화가 아닌 최근 대화에만 집중한다. / 모든 대화를 저장하지 않는다. 

In [7]:
# Conversation Buffer Window 메모리

from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(
    return_messages=True,
    k = 4
)

def add_message(input, output):
    memory.save_context({"input": input}, {"outputs":output})

add_message(1, 1)
add_message(2, 2)
add_message(3, 3)
add_message(4, 4)

memory.load_memory_variables({})

{'history': [HumanMessage(content='1'),
  AIMessage(content='1'),
  HumanMessage(content='2'),
  AIMessage(content='2'),
  HumanMessage(content='3'),
  AIMessage(content='3'),
  HumanMessage(content='4'),
  AIMessage(content='4')]}

In [9]:
add_message(5, 5)
add_message(6, 6)
memory.load_memory_variables({})

{'history': [HumanMessage(content='4'),
  AIMessage(content='4'),
  HumanMessage(content='5'),
  AIMessage(content='5'),
  HumanMessage(content='5'),
  AIMessage(content='5'),
  HumanMessage(content='6'),
  AIMessage(content='6')]}

#### Conversation Summay 메모리
> - llm을 사용, ChatopenAI로부터 chat_model llm을 import -> 비용이 든다.
> - 그대로 저장하는 것이 아닌 요약을 자체적으로 한다.
> - 초반에는 많은 토큰과 저장공간을 차지하지만 후반에는 효과적이다.

In [13]:
from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryMemory(llm=llm)

def add_message(input, output):
    memory.save_context({"input": input}, {"outputs":output})
    
def get_history():
    return memory.load_memory_variables({})

add_message("안녕 나는 쫀떡이야. 나는 대한민국에서 살아", "와 엄청 멋진 이름이야!!")
add_message("쫀떡이는 고양이야", "와 귀엽다")

In [15]:
add_message("쫀떡이는 밥을 너무 좋아해서 별명이 밥생밥사야. 간식도 엄청 좋아해 그래서 토실토실해", "와 귀엽다")
get_history()

{'history': 'The human introduces themselves as 쫀떡 and mentions they live in South Korea. The AI compliments their name as cool and reacts with excitement to the mention of a cat named 밥생밥사야 who loves food and snacks. The AI finds it cute.'}

#### Conversation Summary Buffer 메모리
> - Conversation Summay memory + Conversation Buffer memory
> - 메모리에 보내온 메시지의 수를 저장, limit에 다다른 순간 오래된 메시지들을 요약한다.
> - 가장 최근의 상호작용을 계속 추적 + 오래된 메시지 요약

In [17]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryBufferMemory

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryMemory(llm=llm, max_token_limit=150, return_messages=True)

def add_message(input, output):
    memory.save_context({"input": input}, {"outputs":output})
    
def get_history():
    return memory.load_memory_variables({})

add_message("안녕 나는 쫀떡이야. 나는 대한민국에서 살아", "와 엄청 멋진 이름이야!!")
add_message("쫀떡이는 고양이야", "와 귀엽다")
add_message("쫀떡이는 밥을 너무 좋아해서 별명이 밥생밥사야. 간식도 엄청 좋아해 그래서 토실토실해", "뚱뚱한 고양이는 제일 귀여워!!")
get_history()

{'history': [SystemMessage(content='The human introduces themselves as 쫀떡 and mentions they live in South Korea. The AI compliments the name and reacts with "와 귀엽다" when told that 쫀떡이는 고양이야. 쫀떡이는 밥을 너무 좋아해서 별명이 밥생밥사야. 간식도 엄청 좋아해 그래서 토실토실해. The AI thinks chubby cats are the cutest!')]}

#### Conversation KG 메모리
> - 대화 도중의 엔티티의 knowledge graph를 만든다.
> - 가장 중요한 것들만 뽑아내는 요약본

In [21]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationKGMemory

llm = ChatOpenAI(temperature=0.1)

memory = ConversationKGMemory(llm=llm, return_messages=True)

def add_message(input, output):
    memory.save_context({"input": input}, {"outputs":output})


add_message("안녕 나는 쫀떡이야. 나는 대한민국에서 살아", "와 엄청 멋진 이름이야!!")
add_message("쫀떡이는 고양이야", "와 귀엽다")
add_message("쫀떡이는 밥을 너무 좋아해서 별명이 밥생밥사야. 간식도 엄청 좋아해 그래서 토실토실해", "뚱뚱한 고양이는 제일 귀여워!!")
memory.load_memory_variables({"input": "쫀떡이가 좋아하는 게 뭐야"})

{'history': [SystemMessage(content='On 쫀떡이: 쫀떡이 는 고양이. 쫀떡이 좋아해서 밥. 쫀떡이 별명이 밥생밥사. 쫀떡이 좋아해 간식. 쫀떡이 토실토실해 .')]}

#### Memory on LLMChain
> - LLM chain은 off-the-shelf chain: 일반적인 목적을 가진 chain
> - 빠르게 시작할 수 있다.
> - 스스로 무언가를 만들어볼 때 off-the-shelf chain 보다는 직접 커스텀해서 만든 chain을 활용하는 것이 좋다

In [25]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

# 메모리 클래스가 메모리를 두 가지 방식으로 출력할 수 있다. -> 문자열 형태, message 형태

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=120, memory_key="chat_history")

template = """

    너는 대화를 도와주는 AI야
    
    {chat_history}
    human: {question}
    you: 
"""


chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt= PromptTemplate.from_template(template),
    verbose=True # 프롬프트 로그들을 확인할 수 있다.
)

chain.predict(question="내 이름은 김쫀떡이야")



> Entering new LLMChain chain...
Prompt after formatting:


    너는 대화를 도와주는 AI야
    
    
    human: 내 이름은 김쫀떡이야
    you: 


> Finished chain.


'안녕, 김쫀떡이! 만나서 반가워. 무엇을 도와드릴까?'

In [28]:
chain.predict(question="나는 서울에서 살아")



> Entering new LLMChain chain...
Prompt after formatting:


    너는 대화를 도와주는 AI야
    
    System: The human introduces themselves as Kim Jjeontteogi. The AI responds with a greeting and asks how it can help Kim Jjeontteogi. Kim Jjeontteogi mentions living in Seoul, to which the AI responds positively about the city and offers assistance with any information needed about tourist spots or local culture.
Human: 내 이름이 뭐야
AI: 당신의 이름은 김 쩐떼오기씨로 소개하셨죠. 저는 여러분이 필요로 하는 정보를 제공해드릴 수 있어요. 무엇을 도와드릴까요?
    human: 나는 서울에서 살아
    you: 


> Finished chain.


'AI: 서울은 정말 멋진 도시에요! 관광지나 현지 문화에 대해 궁금한 점이 있으면 언제든지 물어봐주세요. 어떤 도움이 필요하신가요?'

In [30]:
chain.predict(question="내 이름이 뭐야")



> Entering new LLMChain chain...
Prompt after formatting:


    너는 대화를 도와주는 AI야
    
    System: The human introduces themselves as Kim Jjeontteogi. The AI responds with a greeting and offers assistance. Kim Jjeontteogi asks for their name in Korean, to which the AI responds with their name.
Human: 나는 서울에서 살아
AI: AI: 서울은 정말 멋진 도시에요! 관광지나 현지 문화에 대해 궁금한 점이 있으면 언제든지 물어봐주세요. 어떤 도움이 필요하신가요?
Human: 내 이름이 뭐야
AI: 나는 AI야.
    human: 내 이름이 뭐야
    you: 


> Finished chain.


'네 이름은 김 쩐떼오기야.'

In [31]:
memory.load_memory_variables({})

{'chat_history': 'System: Kim Jjeontteogi introduces themselves and the AI responds with a greeting and offers assistance. Kim Jjeontteogi mentions they live in Seoul and the AI expresses admiration for the city, offering help with any questions about tourist spots or local culture.\nHuman: 내 이름이 뭐야\nAI: 나는 AI야.\nHuman: 내 이름이 뭐야\nAI: 네 이름은 김 쩐떼오기야.'}

#### Chat Based Memory

In [33]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=120, memory_key="chat_history", return_messages=True)

prompt = ChatPromptTemplate.from_messages([
    ("system","너는 대화를 통해 인간을 도와주는 AI야"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}")
])


chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt= prompt,
    verbose=True # 프롬프트 로그들을 확인할 수 있다.
)

chain.predict(question="내 이름은 김쫀떡")



> Entering new LLMChain chain...
Prompt after formatting:
System: 너는 대화를 통해 인간을 도와주는 AI야
Human: 내 이름은 김쫀떡

> Finished chain.


'안녕하세요, 김쫀떡님! 무엇을 도와드릴까요?'

In [36]:
# LCEL Based Memory
# 메모리 추가

from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryBufferMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=120, memory_key="chat_history", return_messages=True)

prompt = ChatPromptTemplate.from_messages([
    ("system","너는 대화를 통해 인간을 도와주는 AI야"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}")
])

def load_memory(input):
    # chain에 있는 모든 컴포넌트는 input을 받고 output을 내준다.
    print(input)
    return memory.load_memory_variables({})["chat_history"]

chain = RunnablePassthrough.assign(chat_history=load_memory) | prompt | llm

def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context({"input": question}, {"putput": result.content})
    print(result)

# RunnablePassthrough => 프롬프트가 format되기 전에 우리가 함수를 실행시키는 걸 허락해준다.
# RunnablePassthrough.assign(chat_history=load_memory)
# 실행할 때 가장 먼저 load_memory 함수를 호출 -> 프롬프트가 필요로 하는 chat_history key 내부에 넣는다.(load_memory 함수의 결과값을 chat_history input에 넣는다)

# MessagesPlaceholder(variable_name="chat_history"),
# chat_history가 필요한 프롬프트로 이동

In [37]:
invoke_chain("내 이름은 쫀떡")

{'question': '내 이름은 쫀떡'}
content='안녕, 쫀떡! 무엇을 도와드릴까요?'


In [38]:
invoke_chain("내 이름이 뭐야")

{'question': '내 이름이 뭐야'}
content='당신의 이름은 쫀떡이에요. 저도 기억하고 있어요! 계속 대화를 이어가거나 다른 질문이 있으면 언제든지 물어봐주세요.'
